In [55]:
import random
from deap import base, creator, tools, algorithms

# Definir el problema y los operadores genéticos

# Crear el problema y el tipo de objetivo de aptitud
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

# Inicializar las estructuras de herramientas de DEAP
toolbox = base.Toolbox()

# Definir las operaciones genéticas (selección, crossover, mutación)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", tools.cxPartialyMatched)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)

# Definir el problema y las funciones de inicialización y evaluación del individuo
def init_individual(depot, customers):
    # Generar una solución aleatoria para el VRP
    customer_indices = list(range(len(customers)))
    random.shuffle(customer_indices)
    return [depot] + customer_indices

def evaluate_individual(individual, depot, customers, distances, vehicle_capacity):
    # Evaluar la aptitud de un individuo para el VRP
    num_vehicles = len(individual)
    total_distance = 0.0
    total_demand = 0.0
    for i in range(num_vehicles):
        route = individual[i]
        if len(route) > 2:
            total_distance += distances[depot][route[0]]
            total_demand = customers[route[0]][1]
            for j in range(len(route) - 1):
                total_distance += distances[route[j]][route[j+1]]
                total_demand += customers[route[j+1]][1]
            total_distance += distances[route[-1]][depot]
            if total_demand > vehicle_capacity:
                return float("inf"),
        else:
            total_distance += distances[depot][depot]
    return total_distance,

# Configurar el problema y los parámetros
depot = 0
customers = [(1, 4), (2, 6), (3, 3), (4, 2), (5, 6)]
distances = [
    [0, 2.8, 4.5, 7.2, 5.4, 8.4],
    [2.8, 0, 3.7, 4.4, 7.9, 5.3],
    [4.5, 3.7, 0, 2.8, 4.6, 2.2],
    [7.2, 4.4, 2.8, 0, 5.0, 2.5],
    [5.4, 7.9, 4.6, 5.0, 0, 4.2],
    [8.4, 5.3, 2.2, 2.5, 4.2, 0]
]
vehicle_capacity = 10

# Definir la función de aptitud y la función de distancia
toolbox.register("evaluate", evaluate_individual, depot=depot, customers=customers,
                 distances=distances, vehicle_capacity=vehicle_capacity)

# Definir los operadores genéticos
toolbox.register("individual", init_individual, depot=depot, customers=customers)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Configurar otros parámetros del algoritmo genético (tamaño de la población, número de generaciones, etc.)
population_size = 100
num_generations = 100
cxpb = 0.8
mutpb = 0.2

# Ejecutar el algoritmo genético
population = toolbox.population(n=population_size)

# Definir las estadísticas a recopilar
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("min", min)
stats.register("avg", lambda ind: sum(ind.fitness.values) / len(ind))

# Ejecutar el algoritmo genético
population, logbook = algorithms.eaSimple(
    population,
    toolbox,
    cxpb=cxpb,
    mutpb=mutpb,
    ngen=num_generations,
    stats=stats
)

# Mostrar los resultados finales
best_solution = tools.selBest(population, k=1)[0]
best_distance = evaluate_individual(best_solution, depot, customers, distances, vehicle_capacity)[0]
print("Mejor solución encontrada: ", best_solution)
print("Distancia total de la mejor solución: ", best_distance)
print("Historia de la evolución:")
for gen, evals in logbook:
    print(f"Generación {gen}: {evals}")


AttributeError: 'list' object has no attribute 'fitness'